# Pravděpodobnostní klasifikace (Bayesův klasifikátor)

Úloha zaměřená na implementaci základních pravděpodobnostních metod pro klasifikaci.


## Data
Rozdělení dat uvnitř tříd odpovídá normálnímu rozdělení


In [2]:
import usu
import numpy as np

npzfile = np.load('data/data_11.npz')
npzfile.files


['testData', 'testRef', 'trainData', 'trainRef']

In [3]:
testData = npzfile['testData']
testRef = npzfile['testRef']

trainData = npzfile['trainData']
trainRef = npzfile['trainRef']

nClasses = len(np.unique(trainRef)) 

trainData.shape,trainRef.shape, testData.shape, testRef.shape


((1900, 2), (1900, 1), (100, 2), (100, 1))

### Výpočet úspěšnosti
$$ accuracy = \frac{\text{počet správně klasifikovaných objektů}}{\text{počet všech klasifikovaných  objektů}} $$

Apriorní pravděpodobnost třídy:

$$ P_{ap}(class) = \frac{\text{počet prvků třídy}}{\text{počet všech prvků všech tříd}} $$



Hustota pravděpodobnosti Gaussova rozdělení pro jednorozměrný příznakový vektor:

$$ f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp{-\frac{(x-\mu)^2}{2 \sigma^2}} $$

kde $\mu$ je střední hodnota a $\sigma^2$ je rozptyl

Po logoritmizaci dostaneme:
$$ L(x) = -\frac{1}{2} \log(2 \pi \sigma^2) -\frac{(x-\mu)^2}{2 \sigma^2} $$

Pro soubor N vzorků dat:

$$ L(X) = \sum_{i=0}^{N}{ -\frac{1}{2} \log(2 \pi \sigma^2)} -\frac{1}{2}\sum_{i=0}^{N}{\frac{(x_{i}-\mu)^2}{\sigma^2} } $$



Celkově:

$$ L(class, X) = -\frac{1}{2} \sum_{i=0}^{N}{  \log(2 \pi \sigma_{class}^2)} -\sum_{i=0}^{N}{\frac{(x_{i}-\mu_{class})^2}{2 \sigma_{class}^2} } + \log P_{ap}(class)$$

kde $ Z = -\frac{1}{2} \sum_{i=0}^{N}{ \log(2 \pi \sigma_{class}^2)} + \log P(class) = -\frac{N}{2} \log(2 \pi \sigma_{class}^2) + \log P(class) $ je pak pro každou třídu konstanta a je možné si je předpočítat. 



Výpočet pro vícerozměrný příznakový vektor a plnou i diagonální kovariační matici viz. přednášky ....

In [4]:
# def accuracy(testRef, predRef):
#      return (predRef==testRef).mean() * 100


In [23]:
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.classes = None
        self.class_priors = None
        self.mean_vectors = None
        self.cov_matrices = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        num_classes = len(self.classes)
        num_features = X.shape[1]

        self.class_priors = np.zeros(num_classes)
        self.mean_vectors = np.zeros((num_classes, num_features))
        self.cov_matrices = np.zeros((num_classes, num_features, num_features))

        y = np.array(y, dtype=self.classes.dtype)

        for i, cls in enumerate(self.classes):
            X_cls = X[np.reshape(y == cls, (-1,))]
            self.class_priors[i] = X_cls.shape[0] / X.shape[0]
            self.mean_vectors[i] = np.mean(X_cls, axis=0)
            self.cov_matrices[i] = np.cov(X_cls.T)

    def predict(self, X):
        predictions = []
        for x in X:
            class_probs = []
            for i, cls in enumerate(self.classes):
                mean = self.mean_vectors[i]
                cov = self.cov_matrices[i]
                class_prior = self.class_priors[i]

                exponent = -0.5 * (x - mean).T @ np.linalg.inv(cov) @ (x - mean)
                likelihood = 1 / np.sqrt((2 * np.pi) ** len(x) * np.linalg.det(cov))
                posterior = likelihood * class_prior * np.exp(exponent)

                class_probs.append(posterior)

            predicted_class = self.classes[np.argmax(class_probs)]
            predictions.append(predicted_class)
        return predictions


In [24]:
# Create an instance of the NaiveBayesClassifier
classifier = NaiveBayesClassifier()

# Train the classifier on the training data
classifier.fit(trainData, trainRef)

# Define a new data point for prediction
new_data_point = np.array([1.5, 1.5])

# Predict the class label for the new data point
predicted_label = classifier.predict([new_data_point])

print('Predicted label:', predicted_label)

Predicted label: [1.0]


In [32]:
# Predict the class labels for the test data

predicted_labels = classifier.predict(testData)

# for i in range(len(predicted_labels)):
#     print('Predicted label:', predicted_labels[i], 'True label:', testRef[i])


accuracy = 0
# compute accuraccy
for i in range(len(predicted_labels)):
    if predicted_labels[i] == testRef[i]:
        accuracy += 1

accuracy = accuracy / len(predicted_labels) * 100




print('Accuracy:', accuracy, '%')

Accuracy: 96.0 %
